In [ ]:
# In general, there doesn't appear to be a correlation with location and meteorite landings, except that possibly more
# meteorites are found in areas that are more highly populated. Areas such as the Amazon Rainforest and the Sahara Desert
# seem to have fewer landings, possibly because the population density is much lower in those areas.
# Otherwise, meteorites seem to fall pretty uniformly all over the earth.

In [ ]:
# The last map seems to show more meteorite sightings in modernized locations. Possibly due to increased availability of 
# scientific knowledge/technology in those areas. This map represents the earliest century of reliable data.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import gkey

In [2]:
gmaps.configure(api_key=gkey)

In [3]:
# Read Meteorite_Landings.csv file
meteorite_landings_df = pd.read_csv('Meteorite_Landings.csv')
meteorite_landings_df.head(10)

,name,id,nametype,recclass,mass (g),fall,year,reclat,reclong,GeoLocation
0,Aachen,1,Valid,L5,21.0,Fell,01/01/1880 12:00:00 AM,50.77500,6.08333,"(50.775, 6.08333)"
1,Aarhus,2,Valid,H6,720.0,Fell,01/01/1951 12:00:00 AM,56.18333,10.23333,"(56.18333, 10.23333)"
2,Abee,6,Valid,EH4,107000.0,Fell,01/01/1952 12:00:00 AM,54.21667,-113.00000,"(54.21667, -113.0)"
3,Acapulco,10,Valid,Acapulcoite,1914.0,Fell,01/01/1976 12:00:00 AM,16.88333,-99.90000,"(16.88333, -99.9)"
4,Achiras,370,Valid,L6,780.0,Fell,01/01/1902 12:00:00 AM,-33.16667,-64.95000,"(-33.16667, -64.95)"
5,Adhi Kot,379,Valid,EH4,4239.0,Fell,01/01/1919 12:00:00 AM,32.10000,71.80000,"(32.1, 71.8)"
6,Adzhi-Bogdo (stone),390,Valid,LL3-6,910.0,Fell,01/01/1949 12:00:00 AM,44.83333,95.16667,"(44.83333, 95.16667)"
7,Agen,392,Valid,H5,30000.0,Fell,01/01/1814 12:00:00 AM,44.21667,0.61667,"(44.21667, 0.61667)"
8,Aguada,398,Valid,L6,1620.0,Fell,01/01/1930 12:00:00 AM,-31.60000,-65.23333,"(-31.6, -65.23333)"
9,Aguila Blanca,417,Valid,L,1440.0,Fell,01/01/1920 12:00:00 AM,-30.86667,-64.55000,"(-30.86667, -64.55)"


In [4]:
# Count the row number of the dataset
len(meteorite_landings_df)

45716

In [5]:
# Modify the dataframe to grab "year" - split with " " space
year_split = meteorite_landings_df["year"].str.split(" ", n = 1, expand = True) 
year_split.head()

,0,1
0,01/01/1880,12:00:00 AM
1,01/01/1951,12:00:00 AM
2,01/01/1952,12:00:00 AM
3,01/01/1976,12:00:00 AM
4,01/01/1902,12:00:00 AM


In [6]:
# Modify the year_split to grab only "year" data - split with "/"
date_split = year_split[0].str.split("/", n = 2, expand = True) 
date_split.head()

,0,1,2
0,01,01,1880
1,01,01,1951
2,01,01,1952
3,01,01,1976
4,01,01,1902


In [7]:
# Name the date_split[2] to "Year", and add the data column to meteorite_landings dataframe
meteorite_landings_df["Year"]= date_split[2]
meteorite_landings_df.head()

,name,id,nametype,recclass,mass (g),fall,year,reclat,reclong,GeoLocation,Year
0,Aachen,1,Valid,L5,21.0,Fell,01/01/1880 12:00:00 AM,50.77500,6.08333,"(50.775, 6.08333)",1880
1,Aarhus,2,Valid,H6,720.0,Fell,01/01/1951 12:00:00 AM,56.18333,10.23333,"(56.18333, 10.23333)",1951
2,Abee,6,Valid,EH4,107000.0,Fell,01/01/1952 12:00:00 AM,54.21667,-113.00000,"(54.21667, -113.0)",1952
3,Acapulco,10,Valid,Acapulcoite,1914.0,Fell,01/01/1976 12:00:00 AM,16.88333,-99.90000,"(16.88333, -99.9)",1976
4,Achiras,370,Valid,L6,780.0,Fell,01/01/1902 12:00:00 AM,-33.16667,-64.95000,"(-33.16667, -64.95)",1902


In [8]:
# Delete 'year' column and use 'Year' column for data analyses
del meteorite_landings_df["year"]
meteorite_landings_df.head()

,name,id,nametype,recclass,mass (g),fall,reclat,reclong,GeoLocation,Year
0,Aachen,1,Valid,L5,21.0,Fell,50.77500,6.08333,"(50.775, 6.08333)",1880
1,Aarhus,2,Valid,H6,720.0,Fell,56.18333,10.23333,"(56.18333, 10.23333)",1951
2,Abee,6,Valid,EH4,107000.0,Fell,54.21667,-113.00000,"(54.21667, -113.0)",1952
3,Acapulco,10,Valid,Acapulcoite,1914.0,Fell,16.88333,-99.90000,"(16.88333, -99.9)",1976
4,Achiras,370,Valid,L6,780.0,Fell,-33.16667,-64.95000,"(-33.16667, -64.95)",1902


In [9]:
# Change Year datatypes to integer
meteorite_landings_df["Year"] = pd.to_numeric(meteorite_landings_df["Year"])

In [10]:
# Drop missing data row
clean_meteorite_landings_df = meteorite_landings_df.dropna(how="any")

In [11]:
# Set up a new data frame for Data Analyses
meteorite_map_df = clean_meteorite_landings_df[['name', 'id', 'mass (g)', 'reclat', 'reclong', 'Year']]

In [12]:
# Clean dataframe by dropping any N/A values
clean_meteorite_map = meteorite_map_df.dropna(how='any')
clean_meteorite_map

,name,id,mass (g),reclat,reclong,Year
0,Aachen,1,21.0,50.77500,6.08333,1880.0
1,Aarhus,2,720.0,56.18333,10.23333,1951.0
2,Abee,6,107000.0,54.21667,-113.00000,1952.0
3,Acapulco,10,1914.0,16.88333,-99.90000,1976.0
4,Achiras,370,780.0,-33.16667,-64.95000,1902.0
...,...,...,...,...,...,...
45711,Zillah 002,31356,172.0,29.03700,17.01850,1990.0
45712,Zinder,30409,46.0,13.78333,8.96667,1999.0
45713,Zlin,30410,3.3,49.25000,17.66667,1939.0
45714,Zubkovsky,31357,2167.0,49.78917,41.50460,2003.0


In [13]:
# Create a new data frame for the meteor sightings map
heatmap_df = meteorite_map_df[['mass (g)', 'reclat', 'reclong', 'Year']]

# Create a data frame that eliminates rows with values of 0
heatmap_df = heatmap_df = heatmap_df.loc[heatmap_df['mass (g)'] > 0]

In [14]:
# Set up bins for Years
year_bins = [0, 999.999, 1699.999, 1799.999, 1899.999, 1999.999, 2020]
group_names = ["<1000", "1000-1699", "1700-1799", "1800-1899", "1900-1999", "2000-2020"]

# Add a new column named Year Ramge amd binnning based off the range
heatmap_df["Year Range"] = pd.cut(heatmap_df["Year"], year_bins, labels=group_names)
del heatmap_df["Year"]
heatmap_df

,mass (g),reclat,reclong,Year Range
0,21.0,50.77500,6.08333,1800-1899
1,720.0,56.18333,10.23333,1900-1999
2,107000.0,54.21667,-113.00000,1900-1999
3,1914.0,16.88333,-99.90000,1900-1999
4,780.0,-33.16667,-64.95000,1900-1999
...,...,...,...,...
45711,172.0,29.03700,17.01850,1900-1999
45712,46.0,13.78333,8.96667,1900-1999
45713,3.3,49.25000,17.66667,1900-1999
45714,2167.0,49.78917,41.50460,2000-2020


In [15]:
# Check to see how many meteor sightings there are in each bin
meteor_counts = heatmap_df["Year Range"]
meteor_counts.value_counts()

1900-1999    24003
2000-2020    13386
1800-1899      659
1700-1799       32
1000-1699       14
<1000            1
Name: Year Range, dtype: int64

In [16]:
# Create a dataframe for the most recent bin, years 2000-2020
meteors2000s = heatmap_df.loc[heatmap_df["Year Range"] == "2000-2020", :]

# Store Latitude and Longitude in 'locations'
locations = meteors2000s[['reclat', 'reclong']]
rating = meteors2000s['mass (g)']

# Create Heatmap
# Plot Heatmap
fig = gmaps.figure(zoom_level = 1, center=[0,180])

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=rating, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

# plt.savefig('output/heatmaps_2000s')

Figure(layout=FigureLayout(height='420px'))

In [17]:
# Create a dataframe for the most recent bin, years 2000-2020
meteors1900s = heatmap_df.loc[heatmap_df["Year Range"] == "1900-1999", :]

# Store Latitude and Longitude in 'locations'
locations = meteors1900s[['reclat', 'reclong']]
rating = meteors1900s['mass (g)']

# Create Heatmap
fig = gmaps.figure()

# Plot Heatmap
fig = gmaps.figure(zoom_level = 1, center=[0,180])

# Create Heat Layer
heat_layer = gmaps.heatmap_layer(locations, weights=rating, dissipating=False, max_intensity=10, point_radius=1)

# Add Heat Layer
fig.add_layer(heat_layer)

# Display figure
fig

# plt.savefig('output/heatmap_1900s.png')

Figure(layout=FigureLayout(height='420px'))

In [18]:
# Create a dataframe for the most recent bin, years 2000-2020
meteors1800s = heatmap_df.loc[heatmap_df["Year Range"] == "1800-1899", :]

# Store Latitude and Longitude in 'locations'
locations = meteors1800s[['reclat', 'reclong']]
rating = meteors1800s['mass (g)']

# Plot Heatmap
fig = gmaps.figure(zoom_level = 1, center=[0,180])

# Create Heatmap
fig = gmaps.figure()

# Create Heat Layer
heat_layer = gmaps.heatmap_layer(locations, weights=rating, dissipating=False, max_intensity=10, point_radius=1)

# Add Heat Layer
fig.add_layer(heat_layer)

# Display figure
fig

# plt.savefig('output/heatmap_1800s.png')

Figure(layout=FigureLayout(height='420px'))

In [ ]:
# This map seems to show more meteorite sightings in modernized locations. Possibly due to increased availability of 
# scientific knowledge/technology in those areas.